In [1]:
!pip install plotly panel

  Using cached plotly-6.3.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached bokeh-3.8.0-py3-none-any.whl.metadata (10 kB)
  Using cached linkify_it_py-2.0.3-py3-none-any.whl.metadata (8.5 kB)
  Using cached markdown-3.9-py3-none-any.whl.metadata (5.1 kB)
  Using cached markdown_it_py-4.0.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached mdit_py_plugins-0.5.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached param-2.2.1-py3-none-any.whl.metadata (6.6 kB)
  Using cached pyviz_comms-3.0.6-py3-none-any.whl.metadata (7.7 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached xyzservices-2025.4.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached uc_micro_py-1.0.3-py3-none-any.whl.metadata (2.0 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached plotly-6.3.0-py3-none-any.whl (9.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.1/30.1 MB 921.8 kB/s  0:00:35m0:00:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 715.2 kB/s 

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn

# Read Transaction Data With Categories

In [3]:
# Read transactions_2023_2024_categorized.csv
df = pd.read_csv('transactions_2023_2024_categorized.csv')
# Add year and month columns
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Month Name'] = pd.to_datetime(df['Date']).dt.strftime("%b")
# Remove "Transaction" and "Transaction vs category" columns
df = df.drop(columns=['Transaction', 'Transaction vs category'])
df

,Date,Name / Description,Expense/Income,Amount (VND),Category,Year,Month,Month Name
0,2023-01-05,Rent,Expense,"6,000,000",Housing,2023,1,Jan
1,2023-01-05,Electricity (EVN),Expense,"500,000",Utilities,2023,1,Jan
2,2023-01-05,Water,Expense,"200,000",Utilities,2023,1,Jan
3,2023-01-05,Internet (Viettel),Expense,"250,000",Technology,2023,1,Jan
4,2023-01-05,Phone bill,Expense,"150,000",Communication,2023,1,Jan
...,...,...,...,...,...,...,...,...
322,2024-12-01,Monthly Salary,Income,"21,765,767",Income,2024,12,Dec
323,2024-12-20,Shopping (Shopee/Clothes),Expense,"215,527",Shopping,2024,12,Dec
324,2024-12-16,"Food & Drinks (restaurants, cafes)",Expense,"204,343",Food,2024,12,Dec
325,2024-12-13,Shopping (Shopee/Clothes),Expense,"288,558",Shopping,2024,12,Dec


In [4]:
# For Income rows, assign Name / Description to Category
df['Category'] = np.where(df['Expense/Income'] == 'Income', df['Name / Description'], df['Category'])

# Make Pie Charts - Income / Expense Breakdown

In [ ]:
import pandas as pd
import plotly.express as px
import re

def prepare_amount(df):
    df = df.copy()
    
    if 'Year' not in df.columns:
        df['Date'] = pd.to_datetime(df['Date'])
        df['Year'] = df['Date'].dt.year
    
    df['Amount_num'] = pd.to_numeric(
        df['Amount (VND)'].astype(str).str.replace(r'[^\d\-]', '', regex=True),
        errors='coerce'
    ).fillna(0)
    return df


In [ ]:
def make_pie_chart(df, year, label):  # label: 'Expense' hoặc 'Income'
    df = prepare_amount(df)

    sub_df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]
    total_expense = df[(df['Expense/Income'] == 'Expense') & (df['Year'] == year)]['Amount_num'].sum()
    total_income  = df[(df['Expense/Income'] == 'Income')  & (df['Year'] == year)]['Amount_num'].sum()

    # Pie chart
    pie_fig = px.pie(
        sub_df, values='Amount_num', names='Category',
        color_discrete_sequence=px.colors.qualitative.Set2,
        hole=0.3
    )
    pie_fig.update_traces(textposition='inside', textinfo='label+percent')

    # Text total & saving rate
    if label == 'Expense':
        total_value = total_expense
        saving_text = f" · Saving rate {( (total_income-total_expense)/total_income*100 ):,.1f}% " if total_income > 0 else ""
    else:
        total_value = total_income
        saving_text = ""

    # format 
    total_text = "VND " + f"{int(round(total_value)):,}".replace(",", ".")

    pie_fig.update_layout(
        uniformtext_minsize=10,
        uniformtext_mode='hide',
        title=dict(text=f"{label} Breakdown {year}{saving_text}"),
        annotations=[dict(text=total_text, x=0.5, y=0.5, font_size=12, showarrow=False)]
    )
    return pie_fig


In [9]:
income_pie_fig_2023 = make_pie_chart(df, 2023, 'Income')
expense_pie_fig_2023 = make_pie_chart(df, 2023, 'Expense')


# Make Bar Charts Over Months In A Year

In [17]:
import pandas as pd
import plotly.express as px
import re

def make_monthly_bar_chart(df, year, label):
    d = df.copy()

    # Year / Month / Month Name
    if 'Date' in d.columns:
        d['Date'] = pd.to_datetime(d['Date'])
        d['Year'] = d['Date'].dt.year
        d['Month'] = d['Date'].dt.month
        d['Month Name'] = d['Date'].dt.strftime('%b')

    # Create a numeric column from "Amount (VND)" in case this column is a formatted string
    d['Amount_num'] = pd.to_numeric(
        d['Amount (VND)'].astype(str).str.replace(r'[^\d\-]', '', regex=True),
        errors='coerce'
    ).fillna(0)

    # Filter by year and transaction type (Income/Expense)
    sub = d[(d['Expense/Income'] == label) & (d['Year'] == year)].copy()

    # Total by month
    total_by_month = (sub.groupby(['Month', 'Month Name'])['Amount_num']
                        .sum().to_frame().reset_index()
                        .sort_values('Month').reset_index(drop=True))

    # Difference Income/Expense colors
    color_scale = px.colors.sequential.YlGn if label == 'Income' else px.colors.sequential.OrRd

    # Draw bar chart
    bar_fig = px.bar(
        total_by_month,
        x='Month Name', y='Amount_num',
        text_auto='.2s',
        title=f'{label} per month — {year}',
        color='Amount_num',                # hoặc bỏ hẳn tham số color nếu không cần
        color_continuous_scale=color_scale
    )

    # Format Y (VND)
    bar_fig.update_yaxes(title_text='VND', tickformat=',.0f')   # 1,234,567
   
    return bar_fig


# Putting All Charts Together Into Tabs For 2023/2024

In [18]:
# Pie charts
income_pie_fig_2023 = make_pie_chart(df, 2023, 'Income')
expense_pie_fig_2023 = make_pie_chart(df, 2023, 'Expense')  
income_pie_fig_2024 = make_pie_chart(df, 2024, 'Income')
expense_pie_fig_2024 = make_pie_chart(df, 2024, 'Expense')

# Bar charts
income_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Income')
expense_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Expense')
income_monthly_2024 = make_monthly_bar_chart(df, 2024, 'Income')
expense_monthly_2024 = make_monthly_bar_chart(df, 2024, 'Expense')

# Create tabs
tabs = pn.Tabs(
                        ('2023', pn.Column(pn.Row(income_pie_fig_2023, expense_pie_fig_2023),
                                                pn.Row(income_monthly_2023, expense_monthly_2023))),
                        ('2024', pn.Column(pn.Row(income_pie_fig_2024, expense_pie_fig_2024),
                                                pn.Row(income_monthly_2024, expense_monthly_2024))
                        )
                )
tabs.show()

Launching server at http://localhost:60238


# Create Dashboard

In [19]:
# Dashboard template
template = pn.template.FastListTemplate(
    title='Personal Finance Dashboard',
    sidebar=[pn.pane.Markdown("# Income Expense analysis"), 
             pn.pane.Markdown("Overview of income and expense based on my bank transactions. Categories are obtained using local LLMs."),
             pn.pane.PNG("picture.png", sizing_mode="scale_both")
             ],
    main=[pn.Row(pn.Column(pn.Row(tabs)
                           )
                ),
                ],
    # accent_base_color="#88d8b0",
    header_background="#c0b9dd",
)

template.show()

Launching server at http://localhost:60256
